In [10]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from pandas_ods_reader import read_ods
import pandas as pd
from scoping.models import *
import random
from django.core.mail import send_mail, EmailMessage                                                  
from django.core.management import call_command                                                       
from django.core.validators import validate_email                                                     
import time   

In [2]:
p = Project.objects.get(pk=178)
p

<Project: Regional Impacts Map>

In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupyter-google-e4b72bf44320.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1vp3hSg8mTfxFMtB1UfUEM1Gbi7e8eNCR7AdYabsZ284'
wb = gc.open_by_key(spreadsheet_key)
ws = wb.worksheet("Sheet1")

In [5]:
table = ws.get_all_values()
df = pd.DataFrame(table[1:], columns=table[0])
df.head()

,Email,Name,Surname
0,nicole.vanmaanen@climateanalytics.org,Nicole,van Maanen
1,inga.menke@climateanalytics.org,Inga,Menke
2,burcu.yesil@climateanalytics.org,Burcu,Yesil
3,marina.andrijevic@climateanalytics.org,Marina,Andrijevic
4,michael.hegarty@climateanalytics.org,Mike,Hegarty


In [13]:
for i, row in df.iterrows():
    em = row['Email']                                                                    
    fname = row['Name']                                                                      
    sname = row['Name']                                                            
                                                                                                      
    u, created = User.objects.get_or_create(                                                       
        username=em                                                                                  
    )                                                                                                 
    print(u, created)                                                                                 
    if created:                                                                                                                                                        
        ur, created = ProjectRoles.objects.get_or_create(                                      
            project=p,                                                                            
            user=u,                                                                               
            role='RE'                                                                             
        )                                                                                         
    else:                                                                  
        continue                                                                                             
    pw = call_command('generate_password')                                                            
                                                                                                      
    message = f''' 
Dear {fname},

Thanks for taking part in the regional impacts mapping project.

You can log in here https://apsis.mcc-berlin.net/scoping/user/178

using your email address (lower case) and the following password:                    
                                                                                                      
{pw}   

Thanks a lot,

Max
'''
    u.first_name=fname                                                                                
    u.last_name=sname                                                                                 
    u.email = em                                                                                      
    u.is_superuser = True                                                                             
    u.set_password(pw)                                                                                
                                                                                                      
    u.save()                                                                                          
                                                                                                      
    emessage = EmailMessage(                                                                          
        subject = 'APSIS scoping platform',                                                           
        body = message,                                                                               
        from_email = 'Max Callaghan <callaghan@mcc-berlin.net>',                                      
        to = (em,),                                                                                   
    )                                                                                                 
                                                                                                      
    emessage.send() 
    print(u)
    time.sleep(1)

nicole.vanmaanen@climateanalytics.org True
french_optimizing_miocene_exists


KeyboardInterrupt: 

In [12]:
u.delete()

(3,
 {'admin.LogEntry': 0,
  'auth.User': 1,
  'auth.User_groups': 0,
  'auth.User_user_permissions': 0,
  'scoping.Duplicate': 0,
  'scoping.Note': 0,
  'scoping.Profile': 1,
  'scoping.ProjectRoles': 1,
  'scoping.Query_users': 0,
  'scoping.SnowballingSession_users': 0,
  'scoping.UserTag': 0})